# Lab 5: Выделение признаков символов
### Казахские заглавные буквы

"а ә б в г ғ д е ё ж з и й к қ л м н ң о ө п р с т у ұ ү ф х һ ц ч ш щ ъ ы і ь э ю я"

In [54]:
from PIL import Image as pim, ImageDraw, ImageFont
import os
import matplotlib.pyplot as plt
import csv

In [1]:
def symbol_to_image(symbol, font_name, font_size, output_dir):
    ttfont = ImageFont.truetype(font_name, font_size, encoding="unic")
    mask = ttfont.getmask(symbol)
    text_width, text_height = mask.size

    bottom_padding = 10
    text_height +=  5 * bottom_padding
    
    image = pim.new("L", (text_width, text_height), color="white")
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), symbol, fill="black", font=ttfont)
    
    output_path = output_dir + "/symbols/" + symbol + ".png"
    image.save(output_path)

In [67]:
def get_black_mass(input_image):
    pixels = list(input_image.getdata())
    total_pixels = len(pixels)
    
    black_pixels = sum(1 for pixel in pixels if pixel == 0)
    black_mass = black_pixels / total_pixels

    return black_mass

In [68]:
def get_normalized_center(center_x, center_y, image_width, image_height):
    normalized_center_x = center_x / image_width
    normalized_center_y = center_y / image_height
    
    return normalized_center_x, normalized_center_y

In [69]:
def get_moments_of_inertia(input_image, center_x, center_y):
    W, H = input_image.size
    pixels = list(input_image.getdata())
    
    horizontal_moment = 0
    vertical_moment = 0
    
    for y in range(H):
        for x in range(W):
            pixel = pixels[y * W + x]
            if pixel == 0: 
                horizontal_moment += (y - center_y) ** 2
                vertical_moment += (x - center_x) ** 2
    
    return horizontal_moment, vertical_moment

In [70]:
def get_normalized_moments(horizontal_moment, vertical_moment, image_area):
    normalized_horizontal_moment = horizontal_moment / image_area
    normalized_vertical_moment = vertical_moment / image_area
    
    return normalized_horizontal_moment, normalized_vertical_moment

In [71]:
def get_profiles(input_image):
    W, H = input_image.size
    pixels = list(input_image.getdata())
    x_profile = [0] * W
    y_profile = [0] * H
    
    for y in range(H):
        for x in range(W):
            pixel = pixels[y * W + x]
            if pixel == 0:
                x_profile[x] += 1
                y_profile[y] += 1
    
    return x_profile, y_profile

In [72]:
def get_center_of_mass(input_image):
    W, H = input_image.size
    pixels = list(input_image.getdata())
    
    total_pixels = len(pixels)
    total_x = 0
    total_y = 0

    for y in range(H):
        for x in range(W):
            pixel = pixels[y * W + x]
            if pixel == 0:
                total_x += x
                total_y += y
    
    center_x = total_x / total_pixels
    center_y = total_y / total_pixels
    
    return center_x, center_y

In [73]:
def image_profiles_to_graph(x_profile, y_profile, symbol):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    ax1.bar(range(len(x_profile)), x_profile, color='blue')
    ax1.set_title('Horizontal Profile')
    ax1.set_xlabel('Columns')
    ax1.set_ylabel('Number of black pixels')

    ax2.bar(range(len(y_profile)), y_profile, color='green')
    ax2.set_title('Vertical Profile')
    ax2.set_xlabel('Rows')
    ax2.set_ylabel('Number of black pixels')

    ax1.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    ax2.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    
    fig.tight_layout()

    plt.savefig("output/profiles/" + symbol + "_profile")
    plt.close(fig)


In [79]:
def main():
        font_name = "Times New Roman.ttf"
        font_size = 52

        fields = ['symbol', 'weight', 'center_of_mass', 'moment_of_inertia', 'normalized_moment_of_inertia']

        with open('output/result.csv', mode='a', newline='', encoding="utf-8") as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow(fields)

        output_dir = "output"
        if not os.path.exists(output_dir):
                os.makedirs(output_dir)

        symbols = "аәбвгғдеёжзийкқлмнңоөпрстуұүфхһцчшщъыіьэюя"

        # ITERATE OVER EVERY SYMBOL
        for symbol in symbols:
        # CONVERT SYMBOL TO PNG IMAGE
        symbol_to_image(symbol, font_name, font_size, output_dir)
        
        image = pim.open(output_dir + "/symbols/" + symbol + ".png")
        
        black_mass = get_black_mass(image)
        
        center_x, center_y = get_center_of_mass(image)
        
        image_width, image_height = image.size
        normalized_center_x, normalized_center_y = get_normalized_center(center_x, center_y, image_width, image_height)
        
        horizontal_moment, vertical_moment = get_moments_of_inertia(image, center_x, center_y)
        
        image_area = image_width * image_height
        normalized_horizontal_moment, normalized_vertical_moment = get_normalized_moments(horizontal_moment, vertical_moment, image_area)

        # STORE SYMBOL DATA
        data = [
                str(black_mass),                                    # Wieght
                str((center_x, center_y)),    # Center of Mass
                str((horizontal_moment, vertical_moment)),          # Moment of Inertia
                str((normalized_horizontal_moment,
                        normalized_vertical_moment))                   # Norm. Moment of Inertia
                ]
        
        # CONVERT DATA TO CSV
        with open('output/result.csv', mode='a', newline='', encoding="utf-8") as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow([symbol.split('.')[0], *data])

        #CONVERT PROFILE TO GRAPH
        x_profile, y_profile = get_profiles(image)
        image_profiles_to_graph(x_profile, y_profile, symbol)

if __name__ == "__main__":
    main()